Let's try tuning the prompt for `Qwen/Qwen2.5-0.5B-Instruct`.

In [1]:
from openai import OpenAI 

client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="-"
)

In [2]:
client.models.list()

SyncPage[Model](data=[Model(id='facebook/opt-125m', created=1756744789, object='model', owned_by='vllm', root='facebook/opt-125m', parent=None, max_model_len=256, permission=[{'id': 'modelperm-a0575919121b4cd2a0249ed97c6d6685', 'object': 'model_permission', 'created': 1756744789, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')

In [3]:
def create_chat_completion(
    sys_prompt: str, 
    user_msg: str,
    model: str = 'facebook/opt-125m',
    **kwargs 
):
    res = client.chat.completions.create(
        model=model,
        messages=[
            {
                'role': 'system',
                'content': sys_prompt
            },
            {
                'role': 'user',
                'content': user_msg
            }
        ],
        **kwargs
    )

    return res.choices[0].message.content

Load an exapmle dataset.

In [4]:
from mteb.tasks import NQ

nq = NQ() 
nq.load_data()

/home/oleksii/python/embed_steer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4201/4201 [00:00<00:00, 15229.39 examples/s]


In [6]:
queries = list(nq.queries['test'].items())
corpus = list(nq.corpus['test'].items())
qrels = list(nq.relevant_docs['test'].items())

In [7]:
qrels[:5]

[('test0', {'doc0': 1, 'doc1': 1}),
 ('test1', {'doc6': 1}),
 ('test2', {'doc10': 1}),
 ('test3', {'doc17': 1, 'doc18': 1}),
 ('test4', {'doc42': 1})]

In [8]:
nq.queries['test']['test0']

'what is non controlling interest on balance sheet'

In [9]:
sys_prompt = """
You are a helpful AI assistant. 
Given a document excerpt, generate a concise and highly relevant search question. 
Output only the question in lowercase, without quotes or ending punctuation.
""".strip()

In [10]:
create_chat_completion(
    sys_prompt,
    nq.corpus['test']['doc0'], 
    temperature=0
)

BadRequestError: Error code: 400 - {'error': {'message': 'As of transformers v4.44, default chat template is no longer allowed, so you must provide a chat template if the tokenizer does not define one. None', 'type': 'BadRequestError', 'param': None, 'code': 400}}

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('facebook/opt-125m')

In [14]:
tokenizer.special_tokens_map

{'bos_token': '</s>',
 'eos_token': '</s>',
 'unk_token': '</s>',
 'pad_token': '<pad>'}

In [18]:
tokenizer.tokenize('you are a helpful AI assistant.')

['you', 'Ġare', 'Ġa', 'Ġhelpful', 'ĠAI', 'Ġassistant', '.']

In [15]:
tokenizer.apply_chat_template([{'role': 'user', 'content': 'You are a helpful AI assistant'}])

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [22]:
from jinja2 import Template

# Your template string
template_str = """
{{ (messages|selectattr('role', 'equalto', 'system')|list|last).content|trim if (messages|selectattr('role', 'equalto', 'system')|list) else '' }}

{% for message in messages %}
{% if message['role'] == 'user' %}
### Instruction:
{{ message['content']|trim -}}
{% if not loop.last %}

{% endif %}
{% elif message['role'] == 'assistant' %}
### Response:
{{ message['content']|trim -}}
{% if not loop.last %}

{% endif %}
{% endif %}
{% endfor %}
{% if messages[-1]['role'] != 'assistant' %}
### Response:
{% endif %}
"""

# Sample message history
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
    {"role": "user", "content": "What is the capital of Germany?"},
]

# Compile and render the template
template = Template(template_str)
output = template.render(messages=messages)

print(output)



You are a helpful AI assistant.





### Instruction:
What is the capital of France?





### Response:
The capital of France is Paris.





### Instruction:
What is the capital of Germany?



### Response:

